In [374]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'

In [375]:
train = pd.read_csv('hw3_dectree_train.dat.txt',sep=' ',header=None)
test = pd.read_csv('hw3_dectree_test.dat.txt',sep=' ',header=None)

train = train.rename(columns={0: "x1", 1: "x2", 2:"y"})
test = test.rename(columns={0: "x1", 1: "x2", 2:"y"})

In [376]:
def gini_calculater(df):
    if(len(df)==0):
        return 0
    return 1 - ((len(df[df['y']==1])/len(df))**2 + (len(df[df['y']==-1])/len(df))**2 )

def purify(left,right):
    res = len(left)*gini_calculater(left) + len(right)*gini_calculater(right)
    return res



def decision_stump(df,variables):
    
    res_gini = 100000
    res_separater = 0
    res_feature = ''

    for i in variables:
        df = df.sort_values(by=[i])
        
        for j in range(len(df)):   
            if(j==0):
                separator = (df[i].iloc[j])/2
            else:
                separator = (df[i].iloc[j] + df[i].iloc[j-1])/2
            
            left = df[df[i] < separator].copy()
            right = df[df[i] >= separator].copy()
                


            gini_score = purify(left,right)

            if(gini_score < res_gini):
                res_gini = gini_score
                res_separator = separator
                res_feature = i
    
    
    
    left, right = data_divide(df, res_separator, res_feature)
    
    branch = [left['y'].value_counts().idxmax(),res_separator,res_feature]
        
    return branch, left, right

def data_divide(df,separator,feature):
    
    
    left_df = df[ df[feature] <  separator]
    right_df = df[ df[feature] >=  separator]

    left_df = left_df.reset_index()
    left_df = left_df.drop(columns=["index"])

    right_df = right_df.reset_index()
    right_df = right_df.drop(columns=["index"])
    
    return left_df, right_df
    


def decision_tree(df,variables,prune = False):
    branches = []
    if(prune == True):
        branch, left, right = decision_stump(df,variables)
        branches.append(branch)
        return branches
    else:
        
        if(abs(sum(df['y'])) == len(df)):
            if(len(df) == 0):
                return branches
            else:
                branches.append(df['y'].iloc[0])
                return branches
        else:

            branch, left, right = decision_stump(df,variables)

            branches.append(branch)

            branches.append(decision_tree(left,variables))
            branches.append(decision_tree(right,variables))


            return branches


def fit(x,branches):

    if(len(branches)==3):
        feature = branches[0][2]
        separator = branches[0][1]
        
        if(x[feature] < separator):

            return fit(x,branches[1])
        else:

            return fit(x,branches[2])
   
    else:
        return branches[0]
    

    

def error(df,branches):
    df['y_hat'] = df.apply(lambda row:fit(row[['x1','x2']],branches),axis = 1)
    return len(df[df['y']!=df['y_hat']])/len(df)*100
    


In [377]:
train['y_hat'] = train.apply(lambda row:fit(row[['x1','x2']],branches),axis = 1)

# Q13

In [380]:
variables = ['x1','x2']
branches = decision_tree(train,variables)
branches

[[1, 0.626233, 'x2'],
 [[-1, 0.22443950000000001, 'x1'],
  [[1, 0.11515275, 'x2'], [1], [-1]],
  [[1, 0.541508, 'x1'],
   [[1, 0.3586205, 'x2'],
    [[1, 0.501625, 'x1'], [1], [-1]],
    [[1, 0.26075149999999997, 'x1'], [1], [-1]]],
   [[1, 0.285925, 'x2'], [[1, 0.26603849999999996, 'x2'], [1], [-1]], [1]]]],
 [[-1, 0.8781715000000001, 'x1'], [-1], [1]]]

# Q 14

In [205]:
train['y_hat'] = train.apply(lambda row:fit(row[['x1','x2']],branches),axis = 1)
len(train[train['y_hat'] != train['y']])

0

# Q15

In [204]:
test['y_hat'] = test.apply(lambda row:fit(row[['x1','x2']],branches),axis = 1)
print(len(test[test['y_hat'] != test['y']])/len(test))

0.126


# Q16


In [381]:
g = []
variables = ['x1', 'x2']
Error_list = []

for j in range(300):
    sample = train.sample(frac=1, replace=True)
    branches = decision_tree(sample,variables)
    g.append(branches)
    Error_list.append(error(train,branches))

In [382]:
print(f"{sum(Error_list)/len(Error_list)}%")

5.45%


# Q17

In [383]:
for i in range(len(g)):
    train[i] = train.apply(lambda row:fit(row[['x1','x2']],g[i]),axis = 1)

In [384]:
def sign(x):
    if(x>0):
        return 1
    else:
        return -1

train['predict_random_forect'] = train.apply(lambda row:row[len(g)].sum(),axis=1)
train['predict_random_forect'] = train.apply(lambda row:sign(row['predict_random_forect']),axis = 1)

ratio = len(train[train['predict_random_forect'] != train['y']]) / len(train)*100

print(f"Error rate {ratio}%")

Error rate 0.0%


# Q18

In [385]:
for i in range(len(g)):
    test[i] = test.apply(lambda row:fit(row[['x1','x2']],g[i]),axis = 1)

test['predict_random_forect'] = test.apply(lambda row:row[range(0,len(g))].sum(),axis=1)
test['predict_random_forect'] = test.apply(lambda row:sign(row['predict_random_forect']),axis = 1)

ratio = len(test[test['predict_random_forect'] != test['y']]) / len(test)*100

print(f"Error rate {ratio}%")

Error rate 7.5%


# Q19

In [388]:
train = pd.read_csv('hw3_dectree_train.dat.txt',sep=' ',header=None)
test = pd.read_csv('hw3_dectree_test.dat.txt',sep=' ',header=None)

train = train.rename(columns={0: "x1", 1: "x2", 2:"y"})
test = test.rename(columns={0: "x1", 1: "x2", 2:"y"})

In [389]:
def fit_prune(x,branches):

    
    feature = branches[0][2]
    separator = branches[0][1]
    
    if(x[feature] < separator):
        y = branches[0][0]
    else:
        y = branches[0][0]*-1
        
    return y
   
def error_prune(df,branches):
    df['y_hat'] = df.apply(lambda row:fit_prune(row[['x1','x2']],branches),axis = 1)
    return len(df[df['y']!=df['y_hat']])/len(df)*100


In [390]:
g = []
variables = ['x1', 'x2']
Error_list = []
for i in range(100):
    for j in range(300):
        sample = train.sample(frac=1, replace=True)
        branches = decision_tree(sample,variables,True)
        g.append(branches)
        Error_list.append(error_prune(train,branches))

In [391]:
for i in range(len(g)):
    train[i] = train.apply(lambda row:fit_prune(row[['x1','x2']],g[i]),axis = 1)

In [393]:
train['predict_random_forect'] = train.apply(lambda row:row[range(0,len(g))].sum(),axis=1)
train['predict_random_forect'] = train.apply(lambda row:sign(row['predict_random_forect']),axis = 1)

ratio = len(train[train['predict_random_forect'] != train['y']]) / len(train)*100

print(f"Error rate {ratio}%")

Error rate 12.0%


# Q20

In [397]:
for i in range(len(g)):
    test[i] = test.apply(lambda row:fit_prune(row[['x1','x2']],g[i]),axis = 1)
    
    
    
test['predict_random_forect'] = test.apply(lambda row:row[range(0,len(g))].sum(),axis=1)
test['predict_random_forect'] = test.apply(lambda row:sign(row['predict_random_forect']),axis = 1)

ratio = len(test[test['predict_random_forect'] != test['y']]) / len(test)*100
print(f"Error out {ratio}%")

Error out 15.4%
